```{contents}
```
## API Tools 


API tools expose external capabilities (services, databases, actions) to applications or agents through well-defined HTTP interfaces. In LLM/agent systems, an API tool allows the model to **call a real function** instead of hallucinating results.

**Core idea**

```
LLM / Client  →  API Tool  →  External System  →  Response
```

---

### Why API Tools Are Used

* Access real-time or authoritative data
* Perform actions (create ticket, send email, query DB)
* Enforce deterministic behavior
* Reduce hallucinations
* Enable agentic workflows

---

###  Types of API Tools

####  Read-only Tools

* Search
* Retrieval
* Analytics

####  Action Tools

* Create / Update / Delete
* Trigger workflows
* Execute business logic

#### Hybrid Tools

* Read + act based on conditions

---

### API Tool Anatomy

Every API tool typically defines:

* **Endpoint** (URL)
* **Method** (GET, POST, PUT, DELETE)
* **Input schema**
* **Output schema**
* **Side effects** (optional)

Example:

```
POST /search/tickets
Input : { "query": string }
Output: { "tickets": [...] }
```

---

### Simple API Tool Using FastAPI

#### Tool Definition (Backend)

```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class TicketQuery(BaseModel):
    query: str

@app.post("/search-tickets")
def search_tickets(payload: TicketQuery):
    return {
        "results": [
            {"id": "INC001", "issue": "VPN not working"},
            {"id": "INC002", "issue": "Email access issue"}
        ]
    }
```

This API tool:

* Accepts a query
* Returns structured results
* Can be consumed by humans or agents

---

### Calling the API Tool (Client Side)

```python
import requests

response = requests.post(
    "http://localhost:8000/search-tickets",
    json={"query": "vpn issue"}
)

print(response.json())
```

---

### 7. API Tools in LLM Systems (Conceptual Flow)

![Image](https://miro.medium.com/v2/resize%3Afit%3A2000/1%2A20yoxYdr3daXTHcy8At37Q.png?utm_source=chatgpt.com)

![Image](https://cdn.prod.website-files.com/640f56f76d313bbe39631bfd/650f6755ec1eba8db2438a6b_tool%20use.png?utm_source=chatgpt.com)

![Image](https://images.ctfassets.net/23aumh6u8s0i/4LrNhImjPcdPVpmCIrhlG9/4efed8dfb26630e6cf48b4a478a1f550/genai-tool-calling-intro.png)

1. User asks a question
2. LLM decides a tool is required
3. Tool is called with structured input
4. API returns real data
5. LLM uses response to generate final answer

---

### API Tool as a LangChain Tool

Using API tools inside LangChain:

```python
from langchain.tools import Tool
import requests

def search_tickets(query: str):
    resp = requests.post(
        "http://localhost:8000/search-tickets",
        json={"query": query}
    )
    return resp.json()

ticket_tool = Tool(
    name="TicketSearchTool",
    func=search_tickets,
    description="Search historical IT support tickets"
)
```

Now the LLM can **decide when to call this API**.

---

### API Tools vs Internal Functions

| Aspect              | API Tool | Internal Function |
| ------------------- | -------- | ----------------- |
| Network             | Yes      | No                |
| Scalability         | High     | Limited           |
| Language agnostic   | Yes      | No                |
| Security boundaries | Clear    | Weak              |
| Agent compatibility | Native   | Limited           |

---

### Real-World Demonstration Use Case

**IT Support Agent**

* Tool 1: Search tickets (API)
* Tool 2: Fetch KB article (API)
* Tool 3: Escalate issue (API)

The agent:

1. Retrieves similar incidents
2. Suggests resolution
3. Escalates if confidence < threshold

---